## Sequência de funções valor para os credores

Queremos resolver por indução retroativa um jogo com T períodos.

Estou trabalhando em como deixar esse jogo em torno do parâmetro T

Teremos duas sequências de funções valor: uma para quando apenas o credor sênior propõe e outra para quando apenas o credor júnior propõe. As funções do jogo final serão uma média ponderada destas.



In [1]:
#para mostrar todos os resultados e não apenas o último
from IPython.core.interactiveshell import InteractiveShell

InteractiveShell.ast_node_interactivity = "all"

In [2]:
#libraries
import numpy as np
from numba import jit, jitclass, float64, njit
import matplotlib.pyplot as plt
%matplotlib inline
import quantecon as qe
from scipy.stats import beta

from random import uniform #para a draw da uniforme(0,1)
import math

import time #tempo de computação


### Passo 0: definindo comandos do latex para facilitar a escrita

$%conjunto de informações de s em t$
$\newcommand{\Is}[1]{\theta_{s{#1}}, l_{s{#1}}, l_{j{#1}}}$
$%macros para facilitar a escrita de funções valor$


$%conjunto de informações de j em t$
$\newcommand{\Ij}[1]{\theta_{j{#1}}, l_{s{#1}}, l_{j{#1}}}$

$%função valor de s em t$
$\newcommand{\Ws}[1]{ W_{s{#1}} ( \Is{{#1}}) }$

$%função valor de j em t$
$\newcommand{\Wj}[1]{ W_{j{#1}} ( \Ij{{#1}}) }$

$%operador esperança de s em t. 2 argumentos: o primeiro é o período e o segundo é o termo que ela tira a esperança$

$\newcommand{\Es}[2]{\mathbb{E_{#1}^{s} \big[ {#2}  \mid ( \Is{#1} )    \big] }}$

$%minúsculo não mostra o conjunto de informação$
$\newcommand{\es}[2]{\mathbb{E_{#1}^{s} \big[ {#2}  \big] }}$

$%minúsculo não mostra o conjunto de informação$
$%final b de big para aumentar os parênteses$
$\newcommand{\esb}[2]{\mathbb{E_{#1}^{s} \bigg[ {#2}  \bigg] }}$

$%operador esperança de j em t. 2 argumentos: o primeiro é o período e o segundo é o termo que ela tira a esperança$

$\newcommand{\Ej}[2]{\mathbb{E_{#1}^{j} \big[ {#2}  \mid ( \Ij{#1} )    \big] }}$

$%minúsculo não mostra o conjunto de informação$
$\newcommand{\ej}[2]{\mathbb{E_{#1}^{j} \big[ {#2}  \big] }}$

$%minúsculo não mostra o conjunto de informação$
$%final b para aumentar os parênteses$
$\newcommand{\ejb}[2]{\mathbb{E_{#1}^{j} \bigg[ {#2}  \bigg] }}$


$%comando para usar o máximo com chaves grandes$
$\newcommand{\maximo}[1]{\max \bigg\{ #1 \bigg\}}$


In [3]:
#parâmetros do jogo, apenas para ilustração a princípio



μ = 3 #número de meses entre períodos, não entendi onde entra ainda
ρ = 0.9 # (1 - ρ) é a taxa de depreciação da empresa a cada período
β = 1 #usamos aqui a distribuição Uniforme, depois vamos mudar isto
c0 = 0.05 #custo fixo de ir para a corte
c1 = 0.015 #custo variável de ir para a corte


θ_s0 = 0.28 #habilidade inicial de s
θ_j0 = 0.28 #habilidade inicial de j

In [5]:
#informações que virão dos dados


λj = 0.5 #probabilidade de j propor a cada turno. Não precisaremos estimar, isso virá dos dados

Vmax = 100
L = 20 
L_s = 10
L_j = 10

#valores da dívida de cada jogador (virá dos dados, aqui é exemplo):
Ds = 20
Dj = 10

### Passo 1: número máximo de turnos

Calculado com base nos parãmetros

In [147]:
#função para calcular o máximo de turnos do jogo
def maximo_de_turnos(ρ, Vmax, L):
    
    t = 1
    
    while ρ**t * Vmax > L:
        
        t = t + 1
        
    return t
    

In [148]:
#guardando o número máxim de turnos

T = maximo_de_turnos(ρ, Vmax, L)

T

16

In [149]:
#valor máximo de reorganização da firma a cada período


#sequência de valores da firma para cada período

V = np.empty(T, dtype=np.int_)

for t in range(T):

    V[t] = ρ**t * Vmax

In [150]:
#valor de liquidação


#a dívida total é sempre a soma das dívidas
D = Ds + Dj

#o custo total é uma função do tempo

def C(t):
    Ct = c0 * D + c1 * t * D
    
    return Ct

#e os valores de liquidação também são função do tempo

def Ls(t):
    
    Lst = min(L - C(t), Ds)
    
    return Lst

def Lj(t):
    
    Ljt = min(L - C(t) - Ls(t), Dj)
    
    return Ljt
    
    


### Passo 2: Definir os arrays para guardar as funções valor de cada período

São:

* 100 slots para habilidade do jogador

* 100 slots para o lower bound do adversário

* 100 slots para o lower bound do próprio jogador no próximo período

* 2 slots para dizer se o jogador está propondo ou não
    
* T slots para marcar o período da função valor



In [151]:
#slots para cada habilidade
grid_size = 100


#a vantagem de colocar os dados assim é que se eu quiser teta_s = 0.115, basta procurar θs_vals[114]
θs_vals = np.linspace(0.01, 1, grid_size) 
θj_vals  = np.linspace(0.01, 1, grid_size)  


#teste
# θs_vals[99 - 1]

In [152]:
#arrays para o credor sênior



# primeira entrada é a habilidade verdadeira do jogador
# segunda entrada é lst
# terceira entrada é ljt

# quarta entrada indica se jogador está propondo (1) ou respondendo (0)
# última entrada é o período do jogo

Ws_array = np.zeros((100, 100, 100,2, T))

Wj_array = np.zeros((100, 100, 100,2, T))

#vou deixar a função policy separada porque ela vai ser um resultado da barganha entre os agentes



In [153]:
#uma das sugestões que encontrei no stack overflow foi de dividir a matriz grandona em várias matrizes pequenas
#vou tentar com 100x100x100 primeiro, depois eu adapto o código

# Ws_array1 = np.empty((100, 100, 100,T))
# Ws_array2 = np.empty((100, 100, 100,T))
# Ws_array3 = np.empty((100, 100, 100,T))
# Ws_array4 = np.empty((100, 100, 100,T))
# Ws_array5 = np.empty((100, 100, 100,T))
# Ws_array6 = np.empty((100, 100, 100,T))
# Ws_array7 = np.empty((100, 100, 100,T))
# Ws_array8 = np.empty((100, 100, 100,T))
# Ws_array9 = np.empty((100, 100, 100,T))


Populando o array com os valores de liquidação, que independem das habilidades ou lower bounds



In [154]:
Ws_array[:,:,:, :,(T-1)] = Ls(T)

Wj_array[:,:,:, :,(T-1)] = Lj(T)

In [155]:
#funções para achar os valores nas matrizes

def Ws_find(teta_st, lst, ljt, policy, t):
    
    
    #para achar teta_st = 0.99, buscamos: θs_vals[99 - 1]
    #como os valores serão em casas decimais, temos que multiplicar por 100
    vteta_st = int(100 * teta_st - 1)
    
    vlst = int(100 * lst - 1)
    
    vljt = int(100 * ljt - 1)
    
    return Ws_array[vteta_st, vlst, vljt,policy, t]
    
    

    
def Wj_find(teta_jt, lst, ljt, policy, t):
    
    
    vteta_jt = int(100 * teta_jt - 1)
    
    vlst = int(100 * lst - 1)
    
    vljt = int(100 * ljt - 1)
    
    return Ws_array[vteta_jt, vlst, vljt, policy, t]



def find(y):
    
    x = 100*y - 1
    
    #transformando em int para usar como índice nas matrizes
    
    x = int(x)
    
    return x


### Passo 3: fazer o cálculo das funções valor em T-1, T-2, ..., 1.

### Função para tirar um draw da distribuição Beta


Vamos usar o método da amostragem da inversa da CDF (https://en.wikipedia.org/wiki/Inverse_transform_sampling_method). Outra referência que usei foi: https://blogs.sas.com/content/iml/2013/07/22/the-inverse-cdf-method.html#:~:text=The%20exponential%20distribution%20has%20probability,log(1%E2%80%93u).

A CDF da Beta é 

$$ F_{\beta} ( \theta_{t+1} \mid \theta_{t} ) = 1 - \frac{ (1 - \theta_{t+1})^\beta}{ (1 - \theta_{t})^\beta }, \, \, \theta_{t} \leq \theta_{t+1} \leq 1, \, \beta \geq 1$$

Para invertê-la, basta procurarmos o valor de x tal que $F(x) = u$, onde u é uma retirada da distribuição Uniforme(0,1).

Fazendo os cálculos, esse valor de x é (ou $\theta_{t+1}$, no caso)


$$ \theta_{t+1} =  1 - exp \bigg\{ \frac{1}{\beta} \big[  log (1 - u) + \beta * log(1 - \theta_{t}) \big] \bigg\} $$


In [156]:
#código para tirar draw da distribuição beta

def draw_beta(info_hoje):
    
    if(info_hoje == 1):
        return 1
    else:
    
        u = uniform(0, 1)
        x = 1 - math.exp( (1/β) * (math.log(1-u) + β*math.log(1-info_hoje)) )

        return x




# testando

# draw_beta(0.5)



# #teste com draw da função UNIFORME
# def draw_beta(info_hoje):
#     u = uniform(info_hoje, 1)
#     return u


# # testando
# # draw_beta(0.9)

### Função para tirar o valor esperado do teta amanhã, dada a informação hoje.

Usamos a forma fechada, baseada na esperança de uma variável aleatória truncada: https://en.wikipedia.org/wiki/Truncated_distribution



$$ \mathbb{E} \big[ X \mid X > y \big] =  \frac{\int_{y}^{\infty}  x g(x) dx} {1 - F(y)} $$


Onde:

* $f(x)$ é a pdf da Beta, sem truncar. No caso, usamos a = 1 e b = $\beta$
* $ g(x) = f(x)$ sempre que $x > y$ e 0 caso contrário
* $F(y)$ é a CDF da Beta, sem truncar, avaliada em y


Em nosso caso, y será a informação que temos hoje para formar a expectativa sobre a habilidade amanhã. Na prática, y será ou a habilidade do credor no período atual $\theta_{k,t}, k \in \{s,j\}$ ou o lower bound da habilidade do credor adversário no período atual $l_{k,t}$.

Ilustraremos a fórmula usando $l_{t}$:

<!-- 
$$ \mathbb{E} \big[ \theta_{t+1} \mid \theta_{t+1} > l_t \big] =  \frac{ l_t (1 - l_t)^\beta + \frac{(1-l_t)^{\beta+1} }{(\beta+1)} } {1 - (1 - l_t)^\beta} $$ -->



$$ \mathbb{E} \big[ \theta_{t+1} \mid \theta_{t+1} > l_t \big] =  \frac{ l_t (1 - l_t)^\beta + (1-l_t)^{\beta+1}(\beta+1)^{-1} } {(1 - l_t)^{\beta} } $$



In [157]:
def expec_beta(info_hoje):
    
    #retorna 1 se a info_hoje for 1. A CDF não suporta 1
    if info_hoje == 1:
        return 1
    else:
        num = info_hoje * (1 - info_hoje)**β + ((1-info_hoje)**(β+1))/(β+1)

        denom = (1-info_hoje)**(β)

        return round(num/denom,2)
    

# testando, com beta = 1 e lt = 0.5, deve achar 0.75


# expec_beta(0.5)


# #para os outros valores, não pode ser superior a 1. E tem que ser crescente
# expec_beta(0.3)

# expec_beta(0.9)
# expec_beta(1)

In [158]:
# def expec_beta(info_hoje):
    

#     #para calcular o valor esperado UM PERÍODO À FRENTE, vamos tirar 1000 draws disso e fazer a média
#     #se estiver correto, expec_beta(0.5, 1) deve ser próximo de 0.75, pois Beta = 1 é Uniforme
#     beta_vals = []

#     for t in range(100):
#         beta_amanha = draw_beta(info_hoje)

#         beta_vals.append(beta_amanha)
        
#     return round(np.mean(beta_vals),2)

# #testando
# # expec_beta(0.5)



#teste com draw da função UNIFORME
# def expec_beta(info_hoje):
    
#     summation = 1/2
    
#     resultado = info_hoje/2 + summation
    
#     return round(resultado,2)

#testando
# expec_beta(0.5)

### Como obter o valor esperado da função valor no período seguinte?

Multiplicando a coluna correta de pmf pela coluna correta de Ws_array


Agora que temos a pdf, podemos fazer assim:


$$ \es{t}{ \Ws{t+1} } = \sum_{\theta_{t+1}=0.01}^{1.00} prob(\Is{t+1}) * \Ws{t+1}$$


Na prática, já saberemos $l_{s,t+1}$ e $l_{j,t+1}$, então poderemos fixar esses valores


$$ \es{t}{ \Ws{t+1} } = \sum_{\theta_{t+1}=0.01}^{1.00} prob(\theta_{t+1}) * [ \Ws{t+1} \mid l_{s,t+1}, l_{j,t+1} ]$$


Vamos fazer essa soma usando multiplicação de vetores:

* aproveitaremos que dá pra saber os lower bounds do período seguinte a cada situação, então a única incerteza é sobre teta
* multiplicaremos pmf (vetor linha)
* pelo vetor coluna W correspondente


Como achar a entrada correspondente na matriz?
Por exemplo, achar qual entrada corresponde a lst =  0.58. Basta escrever 57 na entrada correspondente. Ou usar a função find(0.58).




In [162]:
#binning da pdf beta

#gerando 1000 draws

def bin(info_hoje, ndraws):
    
    beta_vals = []

    for k in range(ndraws):
        beta_amanha = draw_beta(info_hoje)

        beta_vals.append(beta_amanha)
    
    #cria os bins e conta quantos valores estão dentro deles
    teta_bins = np.zeros(len(θj_vals))  
    
    for t in range(len(teta_bins)):
        
        #ajustando os bins iniciais e final
        if(t==0):
            pre = 0
        else:
            pre = (θj_vals[t-1]+θj_vals[t])/2
               
        
        if(t==len(teta_bins)-1):
            pos = 1
        else: 
            pos = (θj_vals[t]+θj_vals[t+1])/2
        
        
        for b in beta_vals:
            
            if(b >= pre and b <= pos):
                
                teta_bins[t] += 1
    
    return teta_bins/ndraws
        
        
        
        
#às vezes soma um pouco a mais que um (tipo a sétima casa decimal fica maior que zero, mas ok)
# sum(bin(0.9,1000))




In [163]:
#vamos gerar uma matriz com 100 linhas e 100 colunas
#cada coluna vai representar as probabilidades de teta_amanhã dado teta hoje
#a linha 1 significa que teta_hoje é 0.01
#assim, a linha 1 tem as probabilidades de teta_amanhã dado que teta_hoje é 0.01




#probability mass function
pmf = np.zeros((100,100))

#exemplo para ilustrar
# pmf[0,] = bin(θj_vals[0], β, 1000)


#populando a pmf:

for t in range(len(θj_vals)-1):
    
    pmf[t,:] = bin(θj_vals[t], 1000)

#quais as probabilidades de teta_amanhã se eu sei que teta_hoje = 0.5?

# sum(pmf[54,:])

In [164]:
#teste eliminando valores abaixo de teta_hoje


pmf[54,:]

pmf[54, 54:100]

array([0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   , 0.   ,
       0.015, 0.023, 0.028, 0.019, 0.022, 0.027, 0.014, 0.022, 0.018,
       0.017, 0.025, 0.021, 0.027, 0.019, 0.023, 0.026, 0.018, 0.029,
       0.025, 0.029, 0.019, 0.019, 0.015, 0.027, 0.024, 0.024, 0.014,
       0.021, 0.018, 0.028, 0.025, 0.017, 0.02 , 0.025, 0.02 , 0.025,
       0.02 , 0.022, 0.027, 0.018, 0.022, 0.029, 0.026, 0.022, 0.014,
       0.012])

array([0.015, 0.023, 0.028, 0.019, 0.022, 0.027, 0.014, 0.022, 0.018,
       0.017, 0.025, 0.021, 0.027, 0.019, 0.023, 0.026, 0.018, 0.029,
       0.025, 0.029, 0.019, 0.019, 0.015, 0.027, 0.024, 0.024, 0.014,
       0.021, 0.018, 0.028, 0.025, 0.017, 0.02 , 0.025, 0.02 , 0.025,
       0.02 , 0.022, 0.027, 0.018, 0.022, 0.029, 0.026, 0.022, 0.014,
       0.012])

In [165]:
#valor esperado das funções valor, já organizado

def Ews(teta_hoje, ls_amanha, lj_amanha, t):
    #t é o período para o qual queremos o valor esperado
    #t igual a "t_amanhã"
    
    if t == T:
        return Ls(T)
    else:
    
        teta_hoje = find(teta_hoje)

        ls_amanha = find(ls_amanha)
        lj_amanha = find(lj_amanha)
    
    
        
        Esp_respondendo = sum(pmf[teta_hoje, teta_hoje:100] * Ws_array[teta_hoje:100,ls_amanha, lj_amanha, 0, (t-1) ])
        Esp_propondo = sum(pmf[teta_hoje, teta_hoje:100] * Ws_array[teta_hoje:100,ls_amanha, lj_amanha, 1, (t-1) ])


        Esp = λj * Esp_respondendo + (1 - λj) * Esp_propondo

        return Esp


def Ewj(teta_hoje, ls_amanha, lj_amanha, t):
    
#     breakpoint()
    if t == (T-1):
        return Lj(T)
    else:
        teta_hoje = find(teta_hoje)

        ls_amanha = find(ls_amanha)
        lj_amanha = find(lj_amanha)
    
    
    
        Esp_respondendo = sum(pmf[teta_hoje, teta_hoje:100] * Wj_array[teta_hoje:100,ls_amanha, lj_amanha, 0, (t-1) ])
        Esp_propondo = sum(pmf[teta_hoje, teta_hoje:100] * Wj_array[teta_hoje:100,ls_amanha, lj_amanha, 1, (t-1) ])

        #aqui invertemos a ordem porque j propõe com chance lambda_j
        Esp = λj * Esp_propondo  + (1 - λj) * Esp_respondendo

        return Esp


#teste


# start = time.process_time()


# Ews(0.5, 0.3, 0.2, 16)
# print(time.process_time() - start)

# sum(pmf[49, :] * Ws_array[49,29, 19, 1, 15])


#### Primeiro vamos explicar os cálculos, depois a gente explica a notação de matriz mais adiante


A primeira parte da função valor de s é caso ele venha a propor:


 $$ S_1 (\Is{1}) = \max \bigg\{ L_{s1}, V \es{1}{\theta_s} - \es{1}{W_j}, \es{1}{W_s}
                    \bigg\} $$



In [166]:
#payoff de S quanto ele tem o direito de propor em t

 
#Tem que ter preparado o terreno antes calculando as funções valor no período seguinte

def St(teta_st, lst, ljt, t):
    
    #update do lower bound:
    
    #ls_next é a habilidade dele hoje pois s revela esta quando propõe
    
    ls_next = teta_st
    
      
    
    #quanto s pagará para j neste período? A esperança da função valor de j amanhã, conforme as informações de j hoje.
    
    Pst = Ewj(ljt, lst, ljt, t+1)
    
    
    #lj_next depende do cutoff de j
    
    #algoritmo para calcular cutoff
    
    cjt = ljt
    
    
    #flag é vai parar o loop se demorar muito
    flag = 1
    tol = 0.01
    
    #condição cjt < 0.99 é pra ele não passar de 1
    #veja o que acontece com st(0.28, 0.51, 0.28, 14) se não tiver essa condição
    while (Pst - Ewj(cjt, ls_next, cjt, t+1) > tol and flag < 10000 and cjt < 0.99):
        cjt = cjt + 0.01
        flag = flag + 1
        
    
    #lj_next é o máximo entre o lower bound de j hoje e o cutoff de j
    lj_next = max(ljt, cjt)
    
    #esperança da própria habilidade no próximo período
    θs_next = expec_beta(teta_st)
    
    #OLHAR ESSA PARTE COM CUIDADO, CONFERIR OS ARGUMENTOS DAS FUNÇÕES NO PERÍODO SEGUINTE
    St = [Ls(t), V[t] * θs_next - Pst, Ews(θs_next, ls_next, lj_next, t+1)]
          
    
    policy_St = np.argmax(St)
    
    
    #retorna um vetor com dois elementos: a função valor e a política ótima
    return St[policy_St]


    

    


Quando j propõe é análogo

In [167]:
#payoff de S quanto ele tem o direito de propor em t

 
#Tem que ter preparado o terreno antes calculando as funções valor no período seguinte

def Jt(teta_jt, lst, ljt, t):
    
    #update do lower bound:
    
    #ls_next é a habilidade dele hoje pois s revela esta quando propõe
    
    lj_next = teta_jt
    
      
    
    #quanto s pagará para j neste período? A esperança da função valor de j amanhã, conforme as informações de j hoje.
    
    Pjt = Ews(lst, lst, ljt, t+1)
    
    
    #lj_next depende do cutoff de j
    
    #algoritmo para calcular cutoff
    
    cst = lst
    
    
    #flag é vai parar o loop se demorar muito
    flag = 1
    tol = 0.01
    
    while (Pjt - Ews(cst, cst, lj_next, t+1) > tol and flag < 10000 and cst < 0.99):
        cst = cst + 0.01
        flag = flag + 1
        
    
    #lj_next é o máximo entre o lower bound de j hoje e o cutoff de j
    ls_next = max(lst, cst)
    
    #esperança da própria habilidade no próximo período
    θj_next = expec_beta(teta_jt)
    
    #OLHAR ESSA PARTE COM CUIDADO, CONFERIR OS ARGUMENTOS DAS FUNÇÕES NO PERÍODO SEGUINTE
    Jt = [Lj(t), V[t] * θj_next - Pjt, Ewj(θj_next, ls_next, lj_next, t+1)]
          
    
    policy_Jt = np.argmax(Jt)
    
    
    #retorna um vetor com dois elementos: a função valor e a política ótima
    return Jt[policy_Jt]


    

    


A segunda parte da função valor é se se for chamado a responder:
    
$s_1 (\Is{1}) = $
    $ \esb{1}{ Prob ({j liquidar})  \maximo{L_{s1}, V \es{1}{\theta_s} - L_{j1}} } $
    $ + \esb{1}{ Prob ({j reorganizar})   \maximo{\ej{1}{W_s}, \es{1}{W_s}}    }$
 
 
 Sendo que
 
 $ \Es{1}{Prob (j liq em T-1)}$
 
  $$ = Prob \bigg(L_{j1} > \max \{V \es{1}{ \ej{1}{\theta_j} } - \es{1}{ \ej{1}{W_s} }, \es{1}{ \ej{1}{W_j}  }\} \bigg)$$
  
  
 Repare que $L_{j1} > L{j}$ e que $\es{1}{\theta_{j}}$ cresce em $\theta_{j}$. Então deve existir um threshold de reorganização em t = T-1, $\phi_{j1}$, tal que: 

$$V \phi_{j1} - L_{s} = L{j1} $$

$$ \phi_{j1} = \frac{ L_{j1} + L_{s} }{V} $$

Então se $\es{1}{\theta_{j}} > \phi_{j1}$, j reorganiza a firma. E j liquida a firma caso contrário.


A pergunta é: qual a probabilidade de que $\theta_{j}$ tome um valor menor ou igual a $\phi_{j1}$ ? Isso pode ser respondido usando a \textit{prior} do jogador, a função CDF da distribuição Beta:

$$ \Es{1}{Prob (\textnormal{j liq em T-1})} = F_{\beta} \big( \phi_{j1} \mid l_{j1} \big) $$
 


Na probabilidade de liquidar no período seguinte, temos a seguinte expressão:

$ Prob \bigg( L_{j,t} > \maximo{V_{t+1} \es{t}{\theta_{j,t+1} }  - \es{t}{ \ej{t}{ W_{s,t+1}  } }  ,  \es{t}{ \ej{t}{ W_{j,t+1}  } } } \bigg)$

Onde os termos dentro das expectativas iteradas são calculados com base apenas nos lower bounds de hoje.

1. Vamos considerar o caso em que a esperança de j sobre a sua própria função valor  é o máximo dos argumentos

2. Vamos calcular o threshold para o caso onde ainda pode compensar reorganizar a firma

In [168]:
#vamos criar os componentes da função st

#não preciso me preocupar com if t == T, pois esta condição estará nas funções valor Ws e Wj

def Prob_s(teta_st, lst, ljt, t):
    
    #probabilidade de j liquidar em t, dado ljt 
    #sob a ótica de s, daí o subscrito _s
    
    #caso onde a esperança de j sobre a própria função é o máximo dos argumentos
    if (Ewj(ljt, lst, ljt, t+1) > V[t] * 1 - Ews(lst, lst, ljt, t+1) ):
        
        #se a esperança da função valor amanhã for maior ou igual ao valor de liquidar,
        #a chance de liquidar é zero
        
        if(Ewj(ljt, lst, ljt, t+1) >= Lj(t)):
            return 0
        else: 
            return 1
        
    #caso onde ainda compensa para j reorganizar a firma
    else:

        #threshold para j tentar liquidar
        
        #DÚVIDA SE AQUI EU PRECISO USAR LJ_NEXT E LS_NEXT.
        #ACHO QUE SIM, VOU CONFERIR DEPOIS
        ϕjt = ( Lj(t) + Ews(lst, lst, ljt, t+1) ) / V[t]

        Prob_st = 1 - ((1 - ϕjt)**β)/((1 - ljt)**β)

        #aqui tem que ser no mínimo a igual a zero, apenas por segurança
        Prob_st = max(0, Prob_st)

        return Prob_st



A probabilidade para j é análoga

In [169]:
#vamos criar os componentes da função st

#não preciso me preocupar com if t == T, pois esta condição estará nas funções valor Ws e Wj

def Prob_j(teta_jt, lst, ljt, t):
    
    #probabilidade de s liquidar em t, dado lst 
    #sob a ótica de j, daí o subscrito _j
    
    #caso onde a esperança de s sobre a própria função é o máximo dos argumentos
    if (Ews(lst, lst, ljt, t+1) > V[t] * 1 - Ewj(ljt, lst, ljt, t+1) ):
        
        #se a esperança da função valor amanhã for maior ou igual ao valor de liquidar,
        #a chance de liquidar é zero
        
        if(Ews(lst, lst, ljt, t+1) >= Ls(t)):
            return 0
        else: 
            return 1
        
    #caso onde ainda compensa para j reorganizar a firma
    else:

        #threshold para j tentar liquidar
        
        #DÚVIDA SE AQUI EU PRECISO USAR LJ_NEXT E LS_NEXT.
        #ACHO QUE SIM, VOU CONFERIR DEPOIS
        ϕst = ( Ls(t) + Ewj(ljt, lst, ljt, t+1) ) / V[t]

        Prob_jt = 1 - ((1 - ϕst)**β)/((1 - lst)**β)

        #aqui tem que ser no mínimo a igual a zero, apenas por segurança
        Prob_jt = max(0, Prob_jt)

        return Prob_jt



Payoff de s caso ele responda à uma proposta de liquidação de j

In [170]:
#Payoff de s caso ele responda à uma proposta de liquidação de j

def s_liq(teta_st, lst, ljt, t):
    
    #update dos lower bounds não é necessário aqui
    #é a unica parte da função que independe das funções valor no período seguinte
    
    st_liq = [Ls(t), V[t] * expec_beta(teta_st) - Lj(t) ]

    policy_st_liq = np.argmax(st_liq)


    st_liq = st_liq[policy_st_liq]

    return (st_liq)




Payoff de j caso ele responda à uma proposta de liquidação de s

In [171]:
#Payoff de j caso ele responda à uma proposta de liquidação de s


def j_liq(teta_jt, lst, ljt, t):
    
    #update dos lower bounds não é necessário aqui
    #é a unica parte da função que independe das funções valor no período seguinte
    
    jt_liq = [Lj(t), V[t] * expec_beta(teta_jt) - Ls(t) ]

    policy_jt_liq = np.argmax(jt_liq)


    jt_liq = jt_liq[policy_jt_liq]

    return (jt_liq)




payoff caso s responda à uma proposta de reorganização de j


In [179]:
#payoff caso s responda à uma proposta de reorganização de j

def s_reorg(teta_st, lst, ljt, t):
    #update dos lower bounds
    
    
    #neste caso quem revela a informação é j,
    #vamos ter que colocar o update do lower bound na estrutura da barganha do jogo depois
    lj_next = expec_beta(ljt)
    
    #algoritmo para calcular cutoff
    
    cst = lst
    
    #flag é vai parar o loop se demorar muito
    flag = 1
    tol = 0.01
    #vou usar f() porque a habilidade em f() cresce 0.1 por período
    #se cst crescer 0.1, significa que a função funcionou bem
    
    Pjt = Ews(lst, lst, ljt, t+1)
    
    
    while (Pjt - Ews(cst, cst, lj_next, t+1) > tol and flag < 10000 and cst < 0.99):
        cst = cst + 0.01
        flag = flag + 1
        
    
    #lj_next é o máximo entre o lower bound de j hoje e o cutoff de j
    ls_next = max(lst, cst)
    
    
    
    st_reorg = [Ews(ls_next, ls_next, lj_next, t+1), Ews(teta_st, ls_next, lj_next, t+1)]

    policy_st_reorg = np.argmax(st_reorg)


    st_reorg = st_reorg[policy_st_reorg]
    
    return (st_reorg)



payoff caso j responda à uma proposta de reorganização de s


In [180]:
#payoff caso j responda à uma proposta de reorganização de s

def j_reorg(teta_jt, lst, ljt, t):
    
    #update dos lower bounds
    
    
    #neste caso quem revela a informação é s,
    #vamos ter que colocar o update do lower bound na estrutura da barganha do jogo depois
    ls_next = expec_beta(lst)
    
    #algoritmo para calcular cutoff
    
    cjt = ljt
    
    #flag é vai parar o loop se demorar muito
    flag = 1
    tol = 0.01
    #vou usar f() porque a habilidade em f() cresce 0.1 por período
    #se cst crescer 0.1, significa que a função funcionou bem
    
    Pst = Ewj(ljt, lst, ljt, t+1)
    
    
    while (Pst - Ewj(cjt, ls_next, cjt, t+1) > tol and flag < 10000 and cjt < 0.99):
        cjt = cjt + 0.01
        flag = flag + 1
        
    
    #lj_next é o máximo entre o lower bound de j hoje e o cutoff de j
    lj_next = max(ljt, cjt)
    
    
    
    jt_reorg = [Ewj(lj_next, ls_next, lj_next, t+1), Ewj(teta_jt, ls_next, lj_next, t+1)]

    policy_jt_reorg = np.argmax(jt_reorg)


    jt_reorg = jt_reorg[policy_jt_reorg]
    
    return (jt_reorg)



Finalmente, calculando o valor da função inteira quando os credores estão respondendo a uma proposta

In [181]:
def st(teta_st, lst, ljt, t):
    
    #agregando todas as funções até agora
    
    st = (Prob_s(teta_st, lst, ljt, t)) * s_liq(teta_st, lst, ljt, t) + (1 - Prob_s(teta_st, lst, ljt, t)) * s_reorg(teta_st, lst, ljt, t)
    
    return (st)

In [182]:
def jt(teta_jt, lst, ljt, t):
    
    #agregando todas as funções até agora
    
    jt = (Prob_j(teta_jt, lst, ljt, t)) * j_liq(teta_jt, lst, ljt, t) + (1 - Prob_j(teta_jt, lst, ljt, t)) * j_reorg(teta_jt, lst, ljt, t)
    
    return (jt)

In [144]:
#testando rapidamente os cálculos
#tem que ser período igual a 15 porque é o penúltimo período


# St(0.5,0.4, 0.3, 15)
# Jt(0.5,0.4, 0.3, 15)
# st(0.5,0.4, 0.3, 15)
# jt(0.5,0.4, 0.3, 15)

#todos eles retornam números, então passam no primeiro teste
#depois vamos olhar o código de cada um com cuidado para ver se não tem erro


O que a gente sabe até agora? 

* o valor das funções valor no último período (valor de liquidação)

* calcular as funções valor no penúltimo período, usando St, Jt, st e jt.

Falta apenas a gente popular as matrizes com um loop começando de trás para frente

### Passo 4: populando os arrays usando loops

Vamos relembrar a notação de matriz





#### Como é a notação dessa matriz, na prática?


$ W_{st} (\Is{1})   =$ Ws_array[ $\theta_{st}, l_{st}, l_{jt}, t$] 

Exemplo: 

Ws_array[ $0.5, 0.3,  0.2, 1, 14$] = St(0.5, 0.3,0.2,15)

Traduzindo para St:

* St maiúsculo porque a quarta entrada igual a 1 indica que s propõe

* 0.5, 0.3, 0.2 são os mesmos valores dados pelo conjunto de informação de s

* t = 15 em Ws_array porque a entrada 14 do array se refere ao período 15



Loop para popular as matrizes das funções valor no penúltimo período.

É um ensaio antes de rodar o jogo inteiro

In [176]:
#populando a matriz Ws_Array no penúltimo período quando s propõe

ls_vals = θs_vals
lj_vals = θj_vals


#calculando apenas para valores acima das habilidades iniciais
start = time.process_time()

for i in θs_vals[find(θ_s0):len(θs_vals)]:
    print(time.process_time() - start)
    for j in ls_vals[find(θ_s0):len(ls_vals)]:
        for k in lj_vals[find(θ_j0):len(lj_vals)]:
            Ws_array[find(i), find(j), find(k),1,14] = St(i, j, k, 15)
            Ws_array[find(i), find(j), find(k),0,14] = st(i, j, k, 15)
            
for i in θj_vals[find(θ_j0):len(θj_vals)]:
    print(time.process_time() - start)
    for j in ls_vals[find(θ_s0):len(ls_vals)]:
        for k in lj_vals[find(θ_j0):len(lj_vals)]:
            Wj_array[find(i), find(j), find(k),1,14] = Jt(i, j, k, 15)
            Wj_array[find(i), find(j), find(k),0,14] = jt(i, j, k, 15)
            

            
print("tempo total para popular um período:",time.process_time() - start)

#calculando para teta_s0 = teta_j0 = 0.5 , demorou 15,35s

#calculando para teta_s0 = teta_j0 = 0.28 (foi a menor estimativa feita por Dou et al 2020) , demorou 46s


0.0


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: RuntimeWarning: divide by zero encountered in double_scalars


1.15625
2.1875
3.3125
4.359375
5.515625
6.609375
7.6875
8.796875
9.84375
10.921875
11.9375
13.03125
14.09375
15.15625
16.265625
17.28125
18.328125
19.359375
20.359375
21.390625
22.46875
23.546875
24.546875
25.859375
27.046875
28.40625
29.875
31.1875
32.359375
33.46875
34.84375
36.359375
37.609375
38.859375
40.0625
41.375
42.625
43.90625
44.9375
46.015625
47.140625
48.390625
49.625
51.15625
52.375
53.484375
54.53125
55.546875
56.578125
57.625
58.6875
59.78125
60.96875
62.125
63.34375
64.53125
65.75
66.8125
67.828125
68.859375
69.859375
70.890625
71.90625
72.90625
74.015625
75.0
76.0
77.03125
78.03125
79.09375
80.109375
81.125
81.984375
83.53125
85.1875
86.765625
88.328125
89.921875
91.484375
93.09375
94.65625
96.25
97.8125
99.328125
100.9375
102.4375
104.0
105.53125
107.0625
108.546875
110.078125
111.6875
113.296875
114.796875
116.34375
117.828125
119.34375
120.84375
122.40625
123.875
125.40625
126.9375
128.5
129.984375
131.484375
132.984375
134.5
135.9375
137.5
138.953125
140.5625
142.

In [184]:
#populando a matriz Ws_Array no antepenúltimo período
#t = 14

ls_vals = θs_vals
lj_vals = θj_vals


#calculando apenas para valores acima das habilidades iniciais
start = time.process_time()

for i in θs_vals[find(θ_s0):len(θs_vals)]:
    print(time.process_time() - start)
    for j in ls_vals[find(θ_s0):len(ls_vals)]:
        for k in lj_vals[find(θ_j0):len(lj_vals)]:
            Ws_array[find(i), find(j), find(k),1,13] = St(i, j, k, 14)
            Ws_array[find(i), find(j), find(k),0,13] = st(i, j, k, 14)
            
for i in θj_vals[find(θ_j0):len(θj_vals)]:
    print(time.process_time() - start)
    for j in ls_vals[find(θ_s0):len(ls_vals)]:
        for k in lj_vals[find(θ_j0):len(lj_vals)]:
            Wj_array[find(i), find(j), find(k),1,13] = Jt(i, j, k, 14)
            Wj_array[find(i), find(j), find(k),0,13] = jt(i, j, k, 14)
            

            
print("tempo total para popular um período:",time.process_time() - start)


0.0


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: RuntimeWarning: divide by zero encountered in double_scalars


2.640625
5.296875
7.875
10.515625
13.09375
15.59375
18.1875
20.703125
23.28125
25.859375
28.453125
31.03125
33.640625
36.1875
38.84375
41.390625
43.984375
46.53125
49.359375
51.90625
54.546875
57.09375
60.3125
63.828125
66.59375
70.0
72.609375
75.3125
77.84375
80.4375
82.984375
85.5
88.046875
90.625
93.09375
95.71875
98.234375
100.796875
103.296875
105.828125
108.28125
110.765625
113.265625
115.8125
118.28125
120.84375
123.265625
125.75
128.234375
130.6875
133.203125
135.703125
138.21875
140.71875
143.265625
145.78125
148.3125
150.78125
153.25
155.875
158.921875
161.5
164.234375
166.703125
169.75
173.234375
176.125
178.875
181.84375
184.625
187.046875
189.796875
192.125
193.28125
194.375
195.4375
196.5625
197.75
198.953125
200.0625
201.1875
202.359375
203.546875
204.71875
205.953125
207.125
208.375
209.78125
211.296875
212.765625
214.15625
215.5625
217.296875
218.78125
220.390625
221.96875
223.609375
225.375
227.21875
229.09375
231.0625
233.078125
235.53125
238.09375
240.53125
242.6562

In [185]:
#populando a matriz Ws_Array no ante antepenúltimo período

ls_vals = θs_vals
lj_vals = θj_vals


#calculando apenas para valores acima das habilidades iniciais
start = time.process_time()

for i in θs_vals[find(θ_s0):len(θs_vals)]:
    print(time.process_time() - start)
    for j in ls_vals[find(θ_s0):len(ls_vals)]:
        for k in lj_vals[find(θ_j0):len(lj_vals)]:
            Ws_array[find(i), find(j), find(k),1,12] = St(i, j, k, 13)
            Ws_array[find(i), find(j), find(k),0,12] = st(i, j, k, 13)
            
for i in θj_vals[find(θ_j0):len(θj_vals)]:
    print(time.process_time() - start)
    for j in ls_vals[find(θ_s0):len(ls_vals)]:
        for k in lj_vals[find(θ_j0):len(lj_vals)]:
            Wj_array[find(i), find(j), find(k),1,12] = Jt(i, j, k, 13)
            Wj_array[find(i), find(j), find(k),0,12] = jt(i, j, k, 13)
            

            
print("tempo total para popular um período:",time.process_time() - start)


0.0


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: RuntimeWarning: divide by zero encountered in double_scalars


3.484375
7.265625
10.875
14.40625
17.890625
21.484375
25.6875
29.3125
32.921875
36.359375
39.78125
43.34375
46.84375
50.484375
54.859375
58.828125
62.3125
65.90625
69.5
73.1875
76.8125
80.5
84.265625
88.046875
91.96875
95.921875
99.875
103.84375
107.859375
111.875
115.96875
120.046875
124.125
128.28125
132.328125
136.640625
140.65625
144.78125
148.90625
152.984375
157.0625
161.21875
165.46875
169.703125
173.984375
178.15625
182.375
186.921875
191.25
195.5
199.71875
204.15625
208.46875
212.8125
217.125
221.515625
225.96875
230.453125
234.859375
239.28125
243.84375
248.375
252.84375
257.421875
262.0
266.59375
271.21875
275.859375
280.578125
285.25
290.65625
296.546875
299.453125
302.875
306.203125
309.640625
313.046875
316.640625
320.875
325.53125
328.953125
332.359375
336.0
339.390625
342.8125
347.21875
350.59375
354.09375
357.515625
360.96875
364.515625
367.984375
371.640625
375.21875
378.71875
382.296875
385.953125
389.703125
393.34375
397.0625
400.859375
404.6875
408.625
412.578125
4

In [186]:
#populando as matrizes em todos os períodos do jogo

ls_vals = θs_vals
lj_vals = θj_vals


tempo_total = []

#calculando apenas para valores acima das habilidades iniciais

#range para ir do T = 15 até o t = 1
for t in range(T-1, 0, -1):
    start = time.process_time()

    for i in θs_vals[find(θ_s0):len(θs_vals)]:
#         print(time.process_time() - start)
        for j in ls_vals[find(θ_s0):len(ls_vals)]:
            for k in lj_vals[find(θ_j0):len(lj_vals)]:
                Ws_array[find(i), find(j), find(k),1,(t-1)] = St(i, j, k, t)
                Ws_array[find(i), find(j), find(k),0,(t-1)] = st(i, j, k, t)

    for i in θj_vals[find(θ_j0):len(θj_vals)]:
#         print(time.process_time() - start)
        for j in ls_vals[find(θ_s0):len(ls_vals)]:
            for k in lj_vals[find(θ_j0):len(lj_vals)]:
                Wj_array[find(i), find(j), find(k),1,(t-1)] = Jt(i, j, k, t)
                Wj_array[find(i), find(j), find(k),0,(t-1)] = jt(i, j, k, t)
                
    duração = time.process_time() - start
    tempo_total.append(duração)



    print("tempo total para popular o período", t, ":",time.process_time() - start)



C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: RuntimeWarning: divide by zero encountered in double_scalars


tempo total para popular o período 15 : 205.125
tempo total para popular o período 14 : 324.46875
tempo total para popular o período 13 : 585.5


C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """


tempo total para popular o período 12 : 679.03125
tempo total para popular o período 11 : 648.203125
tempo total para popular o período 10 : 629.71875
tempo total para popular o período 9 : 336.828125
tempo total para popular o período 8 : 292.8125
tempo total para popular o período 7 : 286.265625
tempo total para popular o período 6 : 284.1875
tempo total para popular o período 5 : 289.65625
tempo total para popular o período 4 : 286.28125
tempo total para popular o período 3 : 282.984375
tempo total para popular o período 2 : 278.359375
tempo total para popular o período 1 : 289.109375


In [189]:
#Qual foi o tempo total para rodar esse exemplo?

sum(tempo_total)/60

#quase 95 minutos


94.97552083333333

##### Salvando os arquivos com os arrays

Só para não ter que ficar gerando os dados toda vez

In [199]:
#salvando os arrays para poder trabalhar só na parte da negociação depois


np.savetxt('tempo_total.txt', tempo_total)
np.save('Ws_array', Ws_array)
np.save('Wj_array', Wj_array)
# np.savetxt('Wj_array.npy', Wj_array)



In [201]:
#comando para carregar a base
#aqui tem o final .npy
Ws_array_load = np.load('Ws_array.npy')

Wj_array_load = np.load('Wj_array.npy')

In [202]:
Ws_array_load[:,:,:,:,(T-1)]

Wj_array_load[:,:,:,:,(T-1)]

array([[[[11.3, 11.3],
         [11.3, 11.3],
         [11.3, 11.3],
         ...,
         [11.3, 11.3],
         [11.3, 11.3],
         [11.3, 11.3]],

        [[11.3, 11.3],
         [11.3, 11.3],
         [11.3, 11.3],
         ...,
         [11.3, 11.3],
         [11.3, 11.3],
         [11.3, 11.3]],

        [[11.3, 11.3],
         [11.3, 11.3],
         [11.3, 11.3],
         ...,
         [11.3, 11.3],
         [11.3, 11.3],
         [11.3, 11.3]],

        ...,

        [[11.3, 11.3],
         [11.3, 11.3],
         [11.3, 11.3],
         ...,
         [11.3, 11.3],
         [11.3, 11.3],
         [11.3, 11.3]],

        [[11.3, 11.3],
         [11.3, 11.3],
         [11.3, 11.3],
         ...,
         [11.3, 11.3],
         [11.3, 11.3],
         [11.3, 11.3]],

        [[11.3, 11.3],
         [11.3, 11.3],
         [11.3, 11.3],
         ...,
         [11.3, 11.3],
         [11.3, 11.3],
         [11.3, 11.3]]],


       [[[11.3, 11.3],
         [11.3, 11.3],
         [11.3

array([[[[0., 0.],
         [0., 0.],
         [0., 0.],
         ...,
         [0., 0.],
         [0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.],
         [0., 0.],
         ...,
         [0., 0.],
         [0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.],
         [0., 0.],
         ...,
         [0., 0.],
         [0., 0.],
         [0., 0.]],

        ...,

        [[0., 0.],
         [0., 0.],
         [0., 0.],
         ...,
         [0., 0.],
         [0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.],
         [0., 0.],
         ...,
         [0., 0.],
         [0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.],
         [0., 0.],
         ...,
         [0., 0.],
         [0., 0.],
         [0., 0.]]],


       [[[0., 0.],
         [0., 0.],
         [0., 0.],
         ...,
         [0., 0.],
         [0., 0.],
         [0., 0.]],

        [[0., 0.],
         [0., 0.],
         [0., 0.],
         ...,
   

# Próximos passos



* HIGIENIZAR O CÓDIGO, DEIXAR ORGANIZADO!


* ~debugar o que acontece quando t = 14~
    * bug na função pmf, e acho que vem da função bins() também


* ~criar função de esperança baseada na função beta~
    * expec_beta(info_hoje) retorna o valor esperado de teta amanhã




* ~discretizar a beta para obter a esperança das funções valor no período seguinte~



* ~fazer Js_val~

* ~guardar resultados de Js_val e Ws_val do último período numa  matriz 4D~

* ~pensar numa função Js_next para achar o valor de Js no próximo período baseado nos parâmetros deste período~
    * regra para atualização de lst
    * regra para atualização de ljt
    * expectativa da habilidade no período seguinte

* ~pensar na forma recursiva do jogo~
    * acho que as funções probabilidade também têm que considerar o update do lower bound, pois elas fazem parte do cenário onde o jogador não propõe


* checar todas as funções de cálculo do jogo (Passo 3)


* considerar a estrutura de negociação do jogo

* aumentar a precisão das estimativas do jogo
    * algoritmo para cálculo do cutoff pode ter mais casas decimais
    
    
 * fazer uma estrutura mais enxuta, com uma função que tome como argumento se é s ou j. Ao invés de criar St, Jt, st, jt...
    


* passos finais
    * replicar os gráficos do artigo de referência


### Conferir novamente

1. Se preciso usar ls_next e lj_next nas funções Prob_s e Prob_j

2. Se o valor esperado da função para calcular os thresholds cst e cjt estão corretos

3. A função pmf não está somando um, tem que verificar onde está o erro

